This notebook is used to develop functions for creating pointcloud labels from the cylinder models. Both the offset-vectors as well as the noise-labels are generated

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from fastprogress import progress_bar, master_bar
import os 
import sys
import torch

# Get access to all the files in the repository
cwd = os.getcwd()
parentDir = os.path.dirname( cwd )
sys.path.append(parentDir)

First load a single cylinder model and test the function for finding the closest cylinder

In [2]:
cylinders = pd.read_csv( os.path.join(parentDir, 'data', 'raw', 'QSM', 'detailed', '33_1_000000.csv'), header=0)
print(cylinders.columns)
cylinders.columns = cylinders.columns.str.strip() # Clean whitespaces
print(cylinders.columns)
cylinders.head()

Index(['type', ' ID', ' parentID', ' startX', ' startY', ' startZ', ' endX',
       ' endY', ' endZ', ' radius', ' volume', ' growthVolume', ' length',
       ' growthLength', ' FittingType', 'segmentID', ' parentSegmentID',
       ' segmentMedianRadius', ' segmentGrowthVolume', ' segmentGrowthLength',
       ' branchOrder', ' reverseBranchOrder', ' reversePipeRadiusBranchorder',
       ' reversePipeAreaBranchorder', ' branchID', ' treeID', ' treeSpecies',
       ' translateX', ' translateY', ' translatez', ' gvA', ' gvB', ' gvC',
       ' glA', ' glB', ' glC'],
      dtype='object')
Index(['type', 'ID', 'parentID', 'startX', 'startY', 'startZ', 'endX', 'endY',
       'endZ', 'radius', 'volume', 'growthVolume', 'length', 'growthLength',
       'FittingType', 'segmentID', 'parentSegmentID', 'segmentMedianRadius',
       'segmentGrowthVolume', 'segmentGrowthLength', 'branchOrder',
       'reverseBranchOrder', 'reversePipeRadiusBranchorder',
       'reversePipeAreaBranchorder', 'branchID'

,type,ID,parentID,startX,startY,startZ,endX,endY,endZ,radius,...,treeSpecies,translateX,translateY,translatez,gvA,gvB,gvC,glA,glB,glC
0,cylinder,0,-1,27.268867,23.687374,-1.574510,27.310114,23.677517,-1.342370,0.147356,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
1,cylinder,1,0,27.310114,23.677517,-1.342370,27.323258,23.685699,-1.127219,0.147300,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
2,cylinder,2,1,27.323258,23.685699,-1.127219,27.372589,23.700213,-0.947553,0.147255,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
3,cylinder,3,2,27.372589,23.700213,-0.947553,27.363788,23.667894,-0.756015,0.147214,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0
4,cylinder,4,3,27.363788,23.667894,-0.756015,27.380559,23.656645,-0.605488,0.147182,...,unknownSpecies,0.0,0.0,0.0,0.01,0.401606,0.0,0.01,0.401606,0.0


In [3]:
def closest_cylinder(point, cylinders):
    """
    Find the closest cylinder to a given point in 3D space.

    Parameters:
        point: A 3D point as a numpy array [x, y, z].
        cylinders: A list of dictionaries, where each dictionary represents a cylinder with:
                   - 'start': The start point of the cylinder axis as a numpy array [x, y, z].
                   - 'end': The end point of the cylinder axis as a numpy array [x, y, z].
                   - 'radius': The radius of the cylinder.

    Returns:
        The index of the closest cylinder and the corresponding distance.
    """
    closest_distance = float('inf')
    closest_index = -1
    closet_offset_vector = np.zeros(3)

    for _, cylinder in cylinders.iterrows():
        start = np.array([cylinder['startX'], cylinder['startY'], cylinder['startZ']])
        end = np.array([cylinder['endX'], cylinder['endY'], cylinder['endZ']])
        radius = cylinder['radius']
        id = cylinder['ID']

        # Cylinder axis vector
        axis = end - start
        axis_length = np.linalg.norm(axis)
        axis_unit = axis / axis_length

        # Vector from start point to the given point
        point_vector = point - start

        # Projection of point_vector onto the cylinder axis
        projection_length = np.dot(point_vector, axis_unit)
        projection_point = start + projection_length * axis_unit

        # Clamp the projection point to the cylinder segment
        projection_length_clamped = np.clip(projection_length, 0, axis_length)
        projection_point_clamped = start + projection_length_clamped * axis_unit

        # Compute the distance from the point to the clamped projection point
        distance_to_axis = np.linalg.norm(point - projection_point_clamped)

        # Subtract the cylinder radius to get the distance to the cylinder surface
        distance_to_cylinder_surface = abs(distance_to_axis - radius)

        # Update the closest cylinder
        if distance_to_cylinder_surface < closest_distance:
            closest_distance = distance_to_cylinder_surface
            closest_index = id
            closet_offset_vector = projection_point_clamped - point

    return closest_index, closest_distance, closet_offset_vector

def closest_cylinder_vectorized(point, cylinders):
    # Extract numpy arrays from the DataFrame
    start = cylinders[['startX', 'startY', 'startZ']].to_numpy()
    end = cylinders[['endX', 'endY', 'endZ']].to_numpy()
    radius = cylinders['radius'].to_numpy()
    IDs = cylinders['ID'].to_numpy()

    # Compute axis vectors and lengths
    axis = end - start
    axis_length = np.linalg.norm(axis, axis=1)
    axis_unit = axis / axis_length[:, np.newaxis]

    # Compute vectors from start to the point
    point_vector = point - start

    # Project point_vector onto the cylinder axis
    projection_length = np.sum(point_vector * axis_unit, axis=1)
    projection_point = start + projection_length[:, np.newaxis] * axis_unit

    # Clamp projection to the cylinder segment
    projection_length_clamped = np.clip(projection_length, 0, axis_length)
    projection_point_clamped = start + projection_length_clamped[:, np.newaxis] * axis_unit

    # Compute distances to the cylinder surface
    distances_to_axis = np.linalg.norm(point - projection_point_clamped, axis=1)
    distances_to_surface = np.abs(distances_to_axis - radius)

    # Find the closest cylinder
    closest_idx = np.argmin(distances_to_surface)
    return IDs[closest_idx], distances_to_surface[closest_idx], projection_point_clamped[closest_idx] - point

def closest_cylinder_cuda(point, start, radius, axis_length, axis_unit, IDs, device):
    """
    Find the closest cylinder to a given point using GPU acceleration with PyTorch.
    
    Parameters:
        point: A 3D point as a numpy array [x, y, z].
        cylinders: A pandas DataFrame with cylinder attributes:
                   - 'startX', 'startY', 'startZ'
                   - 'endX', 'endY', 'endZ'
                   - 'radius', 'ID'
    
    Returns:
        The ID of the closest cylinder, the corresponding distance, and the offset vector.
    """
    # Convert point and cylinder data to PyTorch tensors on GPU
    point = torch.tensor(point, dtype=torch.float32, device=device)

    # Vector from start to the point
    point_vector = point - start

    # Projection of point_vector onto the cylinder axis
    projection_length = torch.sum(point_vector * axis_unit, dim=1, keepdim=True)
    # projection_point = start + projection_length * axis_unit

    # Clamp the projection to the cylinder segment
    zero_tensor = torch.zeros_like(projection_length)  # Tensor with the same shape as projection_length, filled with zeros
    projection_length_clamped = torch.clamp(projection_length, zero_tensor, axis_length)
    projection_point_clamped = start + projection_length_clamped * axis_unit

    # Compute distances to the cylinder surface
    distance_to_axis = torch.norm(point - projection_point_clamped, dim=1)
    distance_to_surface = torch.abs(distance_to_axis - radius)

    # Find the closest cylinder
    closest_idx = torch.argmin(distance_to_surface).item()
    closest_distance = distance_to_surface[closest_idx].item()
    closest_offset = (projection_point_clamped[closest_idx] - point).cpu().numpy()

    # Get the ID of the closest cylinder
    closest_id = IDs[closest_idx].item()

    return closest_id, closest_distance, closest_offset

Now load an example pointcloud and perform the cylinder finding for every point

In [ ]:
def generate_offset_cloud_vectorized( cloud, cylinders, masterBar=None ):

    output_data = np.zeros((len(cloud), 7)) # point coordinates, offset vector, cylinder ID

    for i, point in progress_bar(enumerate(cloud), total=len(cloud), master=masterBar):
    # for i, point in enumerate(cloud):
        index, distance, offset_vector = closest_cylinder_vectorized( point, cylinders )

        output_data[i] = np.concatenate( (point, offset_vector, [index]) )

    return output_data

def generate_offset_cloud_cuda( cloud, cylinders, masterBar=None ):

    output_data = np.zeros((len(cloud), 7)) # point coordinates, offset vector, cylinder ID

    if torch.cuda.is_available():
        device = torch.device('cuda')  
        print("Using cuda device")
        # Get the current CUDA device
        device_id = torch.cuda.current_device()
        # Print device properties
        device_name = torch.cuda.get_device_name(device_id)
        print(f"Using CUDA Device: {device_name}")
    else:
        device = torch.device('cpu')
        print("Using cpu")

    start = torch.tensor(cylinders[['startX', 'startY', 'startZ']].values, dtype=torch.float32, device=device)
    end = torch.tensor(cylinders[['endX', 'endY', 'endZ']].values, dtype=torch.float32, device=device)
    radius = torch.tensor(cylinders['radius'].values, dtype=torch.float32, device=device)
    IDs = torch.tensor(cylinders['ID'].values, dtype=torch.int32, device=device)

    # Compute axis vectors and lengths
    axis = end - start
    axis_length = torch.norm(axis, dim=1, keepdim=True)
    axis_unit = axis / axis_length

    for i, point in progress_bar(enumerate(cloud), total=len(cloud), master=masterBar):
    # for i, point in enumerate(cloud):
        index, distance, offset_vector = closest_cylinder_cuda( point, start, radius, axis_length, axis_unit, IDs, device )

        output_data[i] = np.concatenate( (point, offset_vector, [index]) )

    return output_data


In [ ]:
cloudPath = os.path.join( parentDir, "data", "raw", "cloud", "33_1.npy" )
cloud = np.load(cloudPath)

labeled_cloud = generate_offset_cloud_cuda( cloud, cylinders )
print(labeled_cloud[:10,:])

Using cuda device
Using CUDA Device: NVIDIA GeForce RTX 3090


[[ 2.71103306e+01  2.35662594e+01 -1.53770995e+00  1.60905838e-01
   1.20548248e-01 -2.34707594e-02  0.00000000e+00]
 [ 2.71268101e+01  2.35658207e+01 -1.53331006e+00  1.45689011e-01
   1.20685577e-01 -2.07612514e-02  0.00000000e+00]
 [ 2.71269207e+01  2.35607204e+01 -1.52120996e+00  1.47699356e-01
   1.25278473e-01 -2.09234953e-02  0.00000000e+00]
 [ 2.71272507e+01  2.35730896e+01 -1.54901004e+00  1.42509460e-01
   1.14070892e-01 -2.04771757e-02  0.00000000e+00]
 [ 2.71046906e+01  2.35916405e+01 -1.55870998e+00  1.64176941e-01
   9.57336426e-02 -1.57999992e-02  0.00000000e+00]
 [ 2.71079407e+01  2.35954304e+01 -1.54630995e+00  1.61529541e-01
   9.17987823e-02 -2.48025656e-02  0.00000000e+00]
 [ 2.71186791e+01  2.35886002e+01 -1.56051004e+00  1.50188446e-01
   9.87739563e-02 -1.39999390e-02  0.00000000e+00]
 [ 2.70654202e+01  2.36945705e+01 -1.56300998e+00  2.03447342e-01
  -7.19642639e-03 -1.15000010e-02  0.00000000e+00]
 [ 2.71249695e+01  2.35879497e+01 -1.54901004e+00  1.44611359e-0

In [ ]:
def closest_cylinder_cuda_batch(points, start, radius, axis_length, axis_unit, IDs, device):
    """
    Find the closest cylinder to a batch of points using GPU acceleration with PyTorch.
    
    Parameters:
        points: A batch of 3D points as a torch tensor of shape (N, 3).
        start, end, radius, axis_length, axis_unit, IDs: Cylinder data as PyTorch tensors.
        device: CUDA device.
    
    Returns:
        IDs, distances, and offsets for the closest cylinders for each point.
    """
    # Convert points to PyTorch tensors
    points = torch.tensor(points, dtype=torch.float32, device=device)

    # Compute vector from start to points (broadcasting)
    point_vectors = points[:, None, :] - start[None, :, :]  # Shape: (N, M, 3)

    # Projection of point_vector onto the cylinder axis
    projection_lengths = torch.sum(point_vectors * axis_unit[None, :, :], dim=2, keepdim=True)  # Shape: (N, M, 1)

    # Clamp the projection to the cylinder segment
    zero_tensor = torch.zeros_like(projection_lengths)
    projection_lengths_clamped = torch.clamp(projection_lengths, zero_tensor, axis_length[None, :, :])
    projection_points_clamped = start[None, :, :] + projection_lengths_clamped * axis_unit[None, :, :]

    # Compute distances to the cylinder surface
    distances_to_axis = torch.norm(points[:, None, :] - projection_points_clamped, dim=2)  # Shape: (N, M)
    distances_to_surface = torch.abs(distances_to_axis - radius[None, :])  # Shape: (N, M)

    # Find the closest cylinder for each point
    closest_indices = torch.argmin(distances_to_surface, dim=1)  # Shape: (N,)
    closest_distances = distances_to_surface[range(len(points)), closest_indices]  # Shape: (N,)
    closest_offsets = projection_points_clamped[range(len(points)), closest_indices] - points  # Shape: (N, 3)

    # Get the IDs of the closest cylinders
    closest_ids = IDs[closest_indices]

    return closest_ids.cpu().numpy(), closest_distances.cpu().numpy(), closest_offsets.cpu().numpy()

def generate_offset_cloud_cuda_batched(cloud, cylinders, masterBar=None, batch_size=1024):
    output_data = np.zeros((len(cloud), 7))  # point coordinates, offset vector, cylinder ID

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Prepare cylinder data on the GPU
    start = torch.tensor(cylinders[['startX', 'startY', 'startZ']].values, dtype=torch.float32, device=device)
    end = torch.tensor(cylinders[['endX', 'endY', 'endZ']].values, dtype=torch.float32, device=device)
    radius = torch.tensor(cylinders['radius'].values, dtype=torch.float32, device=device)
    IDs = torch.tensor(cylinders['ID'].values, dtype=torch.int32, device=device)
    axis = end - start
    axis_length = torch.norm(axis, dim=1, keepdim=True)
    axis_unit = axis / axis_length

    # Process the cloud in batches
    for i in progress_bar(range(0, len(cloud), batch_size), master=masterBar):
        batch = cloud[i:i + batch_size]
        ids, distances, offsets = closest_cylinder_cuda_batch(batch, start, radius, axis_length, axis_unit, IDs, device)

        # Store results
        output_data[i:i + batch_size, :3] = batch
        output_data[i:i + batch_size, 3:6] = offsets
        output_data[i:i + batch_size, 6] = ids

    return output_data


In [5]:
cloudPath = os.path.join( parentDir, "data", "raw", "cloud", "33_1.npy" )
cloud = np.load(cloudPath)

labeled_cloud = generate_offset_cloud_cuda_batched( cloud, cylinders, batch_size=1024 )
print(labeled_cloud[:10,:])

Using device: cuda


[[ 2.71103306e+01  2.35662594e+01 -1.53770995e+00  1.60905838e-01
   1.20548248e-01 -2.34707594e-02  0.00000000e+00]
 [ 2.71268101e+01  2.35658207e+01 -1.53331006e+00  1.45689011e-01
   1.20685577e-01 -2.07612514e-02  0.00000000e+00]
 [ 2.71269207e+01  2.35607204e+01 -1.52120996e+00  1.47699356e-01
   1.25278473e-01 -2.09234953e-02  0.00000000e+00]
 [ 2.71272507e+01  2.35730896e+01 -1.54901004e+00  1.42509460e-01
   1.14070892e-01 -2.04771757e-02  0.00000000e+00]
 [ 2.71046906e+01  2.35916405e+01 -1.55870998e+00  1.64176941e-01
   9.57336426e-02 -1.57999992e-02  0.00000000e+00]
 [ 2.71079407e+01  2.35954304e+01 -1.54630995e+00  1.61529541e-01
   9.17987823e-02 -2.48025656e-02  0.00000000e+00]
 [ 2.71186791e+01  2.35886002e+01 -1.56051004e+00  1.50188446e-01
   9.87739563e-02 -1.39999390e-02  0.00000000e+00]
 [ 2.70654202e+01  2.36945705e+01 -1.56300998e+00  2.03447342e-01
  -7.19642639e-03 -1.15000010e-02  0.00000000e+00]
 [ 2.71249695e+01  2.35879497e+01 -1.54901004e+00  1.44611359e-0